<a href="https://colab.research.google.com/github/RicardoMiles/Dissertation-template/blob/main/Experiment202411211545.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/SummerProject/src


In [18]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 定义包含CSV文件的文件夹路径
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
data_dir = os.path.join(root_dir, 'Data')
left_data = os.path.join(data_dir,'left')
right_data = os.path.join(data_dir,'right')

folder_path = left_data

# 获取left文件夹中所有CSV文件的列表
csv_files_left = [os.path.join(left_data, file) for file in os.listdir(left_data) if file.endswith('.csv')]
# 获取right文件夹中所有CSV文件的列表
csv_files_right = [os.path.join(right_data, file) for file in os.listdir(right_data) if file.endswith('.csv')]

print(f"Number of left files: {len(csv_files_left)}")
print(f"Number of right files: {len(csv_files_right)}")

Number of left files: 26
Number of right files: 26


In [19]:
from scipy import stats
# 定义特征提取函数
def extract_features(group):
    features = []
    # 遍历第三列到最后一列
    for col in group.columns[2:]:  # 从第三列开始提取特征
        data = group[col].values
        features.append(data.mean())  # 均值
        features.append(data.std())   # 标准差
        features.append(stats.skew(data))  # 偏度
        features.append(stats.kurtosis(data))  # 峰度
    return features

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime  # 导入datetime模块来获取当前时间

# 初始化CSV文件计数器
csv_files_count = 0
all_features = []  # 确保这个列表在代码的其他地方被初始化
all_labels = []  # 确保这个列表在代码的其他地方被初始化

# 处理left文件夹中的CSV文件
for file_name in csv_files_left:
    df = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped = df.groupby(df.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped:
        # 过滤掉time小于0的数据
        group = group[group[df.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            # features = extract_features(group)
            all_features.append(features)
            # 获取标签
            all_labels.append('left')

# 处理right文件夹中的CSV文件
for file_name in csv_files_right:
    df_right = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped_right = df_right.groupby(df_right.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped_right:
        # 过滤掉time小于0的数据
        group = group[group[df_right.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            # features = extract_features(group)
            all_features.append(features)
            # 获取标签
            all_labels.append('right')

# 将特征和标签转换为NumPy数组
features_array = np.array(all_features)
labels_array = np.array(all_labels)

# 打印特征数组的形状
print(f"Features array shape: {features_array.shape}")

# 打印标签数组的形状
print(f"Labels array shape: {labels_array.shape}")

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

# 打印处理的CSV文件数量
print(f"Total CSV files processed: {csv_files_count}")

Features array shape: (7220, 81, 101)
Labels array shape: (7220,)
Current processing time: 2024-11-21 15:33:48
Total CSV files processed: 52


In [22]:
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Current processing time: 2024-11-21 15:33:54


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 初始化标签编码器
label_encoder = LabelEncoder()

# 将训练集和测试集的标签转换为数值
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# 确保转换后的标签是正确的数据类型
y_train_encoded = y_train_encoded.astype('int32')
y_test_encoded = y_test_encoded.astype('int32')

In [27]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 假设 X_train, X_test, y_train, y_test 已经定义并准备好了
# 构建LSTM模型
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=25, return_sequences=True))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(LSTM(units=25))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(Dense(1, activation='sigmoid'))  # 二分类问题

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 早停法
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 训练模型
model.fit(X_train, y_train_encoded, epochs=50, batch_size=128, validation_data=(X_test, y_test_encoded))

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Epoch 1/50
46/46 [==============================] - 8s 97ms/step - loss: 0.7027 - accuracy: 0.5196 - val_loss: 0.6911 - val_accuracy: 0.5229
Epoch 2/50
46/46 [==============================] - 4s 82ms/step - loss: 0.6936 - accuracy: 0.5289 - val_loss: 0.6924 - val_accuracy: 0.5180
Epoch 3/50
46/46 [==============================] - 4s 77ms/step - loss: 0.6874 - accuracy: 0.5440 - val_loss: 0.6938 - val_accuracy: 0.5083
Epoch 4/50
46/46 [==============================] - 4s 77ms/step - loss: 0.6853 - accuracy: 0.5530 - val_loss: 0.6965 - val_accuracy: 0.5125
Epoch 5/50
46/46 [==============================] - 4s 79ms/step - loss: 0.6791 - accuracy: 0.5715 - val_loss: 0.6963 - val_accuracy: 0.5249
Epoch 6/50
46/46 [==============================] - 4s 78ms/step - loss: 0.6726 - accuracy: 0.5907 - val_loss: 0.7018 - val_accuracy: 0.5222
Epoch 7/50
46/46 [==============================] - 4s 77ms/step - loss: 0.6665 - accuracy: 0.6020 - val_loss: 0.7103 - val_accuracy: 0.5242
Epoch 8/50
46

In [30]:
# 预测测试集
predictions = model.predict(X_test)

# 将概率转换为类别
predicted_classes = (predictions > 0.5).astype("int32")

# 打印预测结果
print(predicted_classes)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

46/46 [==============================] - 1s 13ms/step
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [1]]
Current processing time: 2024-11-21 15:45:02


In [31]:
# 计算精确度、召回率和F1分数
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(y_test_encoded, predicted_classes)
recall = recall_score(y_test_encoded, predicted_classes)
f1 = f1_score(y_test_encoded, predicted_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 计算混淆矩阵
cm = confusion_matrix(y_test_encoded, predicted_classes)
print("Confusion Matrix:")
print(cm)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Precision: 0.47341115434500647
Recall: 0.5244252873563219
F1 Score: 0.49761417859577367
Confusion Matrix:
[[342 406]
 [331 365]]
Current processing time: 2024-11-21 15:45:04
